# $\mathsf{PyMWGCprogen}$ code

### Import standard Python packages

In [1]:
#!/usr/bin/env python
# coding: utf-8
from __future__ import division
import sys
import matplotlib.pyplot as plt
import numpy as np
import pylab as pt
from pylab import *
import time
import random
from math import *
from scipy.optimize import fsolve

### Import galpy function

In [2]:
from astropy import units
from galpy.potential import MWPotential2014, ChandrasekharDynamicalFrictionForce, NFWPotential, HernquistPotential, MN3ExponentialDiskPotential, DehnenSphericalPotential
from galpy.potential import rtide, evaluaterforces, evaluater2derivs
from galpy.potential.mwpotentials import McMillan17, Irrgang13I
from galpy.orbit import Orbit
from galpy.util import conversion
from galpy.potential import plotRotcurve
from galpy.potential import MovingObjectPotential
from galpy.potential import NonInertialFrameForce
from galpy.potential import (evaluateRforces, evaluatephitorques,evaluatezforces)
from galpy.potential import vesc

A new version of galpy (1.10.0) is available, please upgrade using pip/conda/... to get the latest features and bug fixes!


### Import colossus function

In [3]:
from colossus.cosmology import cosmology
cosmology.setCosmology('planck15');
from colossus.halo import concentration
from colossus.halo import profile_hernquist

In [4]:
# Debut du decompte du temps  ############################
start_time = time.time()

### Useful fonctions

In [5]:
def Average(lst): 
    return sum(lst) / len(lst) 

### Initializing properties of dwarfs

In [6]:
tab = ['LMC', 'SMC', 'Sgr', 'Fornax', 'Carina', 'Leo II', 'Sextans', 'Sculptor', 'Draco', 'Umi', 'Leo I']
Md0=[198.8e9,77.3e9,6.2e10,21.9e9,0.8e9,1.6e9,2e9,5.7e9,3.5e9,2.8e9,5.6e9] # Pre-infall mass from Read 2019 (see https://ui.adsabs.harvard.edu/abs/2019MNRAS.487.5799R/abstract)
#Md0=[1e11,2.6e10,1.4e10,0.79e9,0.398e9,0.316e9,0.316e9,1.99e9,3.16e9,0.79e9,1.99e9] #Current mass from Errani, Peñarrubia & Walker 2018 (see https://ui.adsabs.harvard.edu/abs/2018MNRAS.481.5073E/abstract) for the cuspy (ga=1) DM halo of dwarfs.

print(Md0)

sc=[] #list of scale radii for dwarf halos
hmr=[] #list of half-mass radii for dwarf halos
redshift=0 # redshit for the scale radii
print(Md0[0])

######## Calculating the scale radius ##################

for kk1 in Md0:
	Mv=kk1*0.7
	C1=concentration.concentration(Mv,'vir',0,model ='prada12')
	p_hern = profile_hernquist.HernquistProfile(M = Mv, c = C1, z = redshift, mdef = 'vir')
	sc.append(p_hern.par['rs']/0.7)
	hmr.append(p_hern.par['rs']/(0.7*(sqrt(2)-1)))

##########################

print(hmr)
print(sc)

[198800000000.0, 77300000000.0, 62000000000.0, 21900000000.0, 800000000.0, 1600000000.0, 2000000000.0, 5700000000.0, 3500000000.0, 2800000000.0, 5600000000.0]
198800000000.0
[29.120973475224666, 19.632520537167565, 17.91105644169623, 11.635564892019563, 3.007538411847519, 3.983063357388343, 4.361288114872841, 6.688070732143269, 5.47866912642692, 5.00186109631168, 6.639759087965385]
[12.06230216294522, 8.132056270063128, 7.4190024945805675, 4.819608784146741, 1.2457631995452816, 1.649838862421566, 1.8065046865969199, 2.770289603364298, 2.2693390559207876, 2.0718387031986563, 2.7502782651252753]


### Initializing properties of GCs

In [8]:
fname = 'dataVasiliev21.txt'
dtype1 = np.dtype([('name', '|S20'),('alpha', 'f8'), ('delta', 'f8'), ('distance', 'f8'), ('Vlos', 'f8'), ('eVlos', 'f8'), ('mualpha', 'f8'), ('mudelta', 'f8'),('emualpha', 'f8'),('emudelta', 'f8'),('covmu', 'f8')])
dataGCs = np.loadtxt(fname, dtype=dtype1, usecols=(0,1,2,3,4,5,6,7,8,9,10))
	
AAA=['NGC104' ,'NGC288','NGC362','Whiting1', 'NGC1261', 'Pal1' ,'E1', 'Eridanus',
 'Pal2', 'NGC1851', 'NGC1904' ,'NGC2298' ,'NGC2419', 'Pyxis' ,'NGC2808' ,'E3',
 'Pal3' ,'NGC3201', 'Pal4', 'Crater', 'NGC4147' ,'NGC4372', 'Rup106' ,'NGC4590',
 'NGC4833' ,'NGC5024', 'NGC5053', 'NGC5139' ,'NGC5272' ,'NGC5286', 'NGC5466',
 'NGC5634', 'NGC5694', 'IC4499' ,'NGC5824' ,'Pal5', 'NGC5897' ,'NGC5904', 'NGC',
 'NGC5946' ,'BH176', 'NGC5986', 'FSR1716', 'Pal14', 'BH184' ,'NGC6093', 'NGC6121',
 'NGC6101', 'NGC6144' ,'NGC6139' ,'Terzan3', 'NGC6171', 'ESO45211', 'NGC6205',
 'NGC6229' ,'NGC6218' ,'FSR1735' ,'NGC6235', 'NGC6254', 'NGC6256' ,'Pal15',
 'NGC6266', 'NGC6273', 'NGC6284', 'NGC6287', 'NGC6293', 'NGC6304' ,'NGC6316',
 'NGC6341' ,'NGC6325', 'NGC6333', 'NGC6342' ,'NGC6356', 'NGC6355' ,'NGC6352',
 'IC1257' ,'Terzan2', 'NGC6366' ,'Terzan4', 'BH229', 'NGC6362' ,'NGC6380',
 'Terzan1' ,'Ton2' ,'NGC6388', 'NGC6402' ,'NGC6401', 'NGC6397', 'Pal6' ,'NGC6426',
 'Djorg1', 'Terzan5', 'NGC6440' ,'NGC6441', 'Terzan6' ,'NGC6453', 'NGC6496',
 'Terzan9' ,'Djorg2' ,'NGC6517' ,'Terzan10' ,'NGC6522', 'NGC6535', 'NGC6528',
 'NGC6539' ,'NGC6540', 'NGC6544' ,'NGC6541', 'ESO28006' ,'NGC6553', 'NGC6558',
 'Pal7' ,'Terzan12', 'NGC6569', 'BH261', 'NGC6584', 'NGC6624' ,'NGC6626',
 'NGC6638' ,'NGC6637' ,'NGC6642', 'NGC6652' ,'NGC6656' ,'Pal8', 'NGC6681',
 'NGC6712' ,'NGC6715' ,'NGC6717' ,'NGC6723' ,'NGC6749' ,'NGC6752', 'NGC6760',
 'NGC6779' ,'Terzan7', 'Pal10' ,'Arp2', 'NGC6809' ,'Terzan8' ,'Pal11' ,'NGC6838',
 'NGC6864' ,'NGC6934' ,'NGC6981' ,'NGC7006' ,'NGC7078' ,'NGC7089', 'NGC7099',
 'Pal12', 'Pal13', 'NGC7492']

### Galpy parameters

In [9]:
###### Integration time and timescale ###########################

ts= np.linspace(0,-10,2000)*units.Gyr # Integrations over 10 Gyr
ts1= np.linspace(0,-10,2000)

####### MW potential ##########################

MWPotential2014[2]*= 1.5

### Code parameters

In [10]:
Na=2 # Na is the number of GC Monte Carlo samples. Try Na=2 for a first test. We use Na=1000 in the paper. 
KR=1 # KR is the number of dwarf Monte Carlo samples. Try KR=1 for a first test. We use KR=100 in the paper. 
# We ran (Na,KR)=(1000,100) on a 256 CPU nodes of a cluster.

### Running the code

In [ ]:
for kr in range(0,KR,1):
	
	print('kr',kr)

	my_dwarf_pots2 = {}
	my_dwarf_orbits2 = {}
	my_dwarf_orbpots2 = {}

	i2=0
	name2='LMC'

	IG0=np.loadtxt('Orbmax/{}ORB.txt'.format(i2))
	dwarf_orbit2=Orbit(IG0[kr],radec=True)
	
	cdf2= ChandrasekharDynamicalFrictionForce(GMs=Md0[i2]*units.Msun,rhm=hmr[i2]*units.kpc,dens=MWPotential2014) #with dynamical friction
	dwarf_orbit2.integrate(ts,MWPotential2014+cdf2,method='dop853_c')
	
	dwarf_pot2=HernquistPotential(amp=2*Md0[i2]*units.Msun,a=sc[i2]*units.kpc)
	my_dwarf_pots2[name2]= dwarf_pot2
	
	my_dwarf_orbpots2[name2]= MovingObjectPotential(dwarf_orbit2,pot=dwarf_pot2)
	my_dwarf_orbits2[name2]= dwarf_orbit2
		
	olmc=my_dwarf_orbits2['LMC']
	moving_lmcpot = my_dwarf_orbpots2['LMC']
	print('moving lmc potential created')

	loc_origin= 1e-4 # Small offset in R to avoid numerical issues

	ax= lambda t: evaluateRforces(moving_lmcpot,loc_origin,0.,phi=0.,t=t,use_physical=False)
	ay= lambda t: evaluatephitorques(moving_lmcpot,loc_origin,0.,phi=0.,t=t,use_physical=False)/loc_origin
	az= lambda t: evaluatezforces(moving_lmcpot,loc_origin,0.,phi=0.,t=t,use_physical=False)

	t_intunits= olmc.time(use_physical=False)[::-1] # need to reverse the order for interp
	ax4int= [ax(t) for t in t_intunits]
	ax_int= lambda t: np.interp(t,t_intunits,ax4int)
	ay4int= [ay(t) for t in t_intunits]
	ay_int= lambda t: np.interp(t,t_intunits,ay4int)
	az4int= [az(t) for t in t_intunits]
	az_int= lambda t: np.interp(t,t_intunits,az4int)

	nip= NonInertialFrameForce(a0=[ax_int,ay_int,az_int])

	print('acceleration by LMC calculated')
	
	my_dwarf_pots = {}
	my_dwarf_orbits = {}
	my_dwarf_orbpots = {}
		
	for i,name in enumerate(tab):
		
		if i==0:
			continue

		print('Dwarf number',i,'/11')

		IG0=np.loadtxt('Orbmax/{}ORB.txt'.format(i))
		dwarf_orbit=Orbit(IG0[kr],radec=True)
		
		cdf= ChandrasekharDynamicalFrictionForce(GMs=Md0[i]*units.Msun,rhm=hmr[i]*units.kpc,dens=MWPotential2014) #with dynamical friction
		dwarf_orbit.integrate(ts,MWPotential2014 + nip  + cdf,method='dop853_c')
		
		dwarf_pot=HernquistPotential(amp=2*Md0[i]*units.Msun,a=sc[i]*units.kpc)
		my_dwarf_pots[name]= dwarf_pot
		
		my_dwarf_orbpots[name]= MovingObjectPotential(dwarf_orbit,pot=dwarf_pot)
		my_dwarf_orbits[name]= dwarf_orbit
		
	print('moving potentials for dwarfs created')
    

################### Sampling ###################
	Number=[]
	NAME=[]
	PERCENT=[]
	ACCRETT=[]
	ACCRETTV=[]
	
	for ff,name2 in enumerate(AAA):

		Rai=dataGCs['alpha'][ff]
		Deci=dataGCs['delta'][ff]

		pmrai=dataGCs['mualpha'][ff]
		pmdeci=dataGCs['mudelta'][ff]
		
		epmrai=dataGCs['emualpha'][ff]
		epmdeci=dataGCs['emudelta'][ff]
		ecovmu=dataGCs['covmu'][ff]
		
		mean = [pmrai, pmdeci]
		cov = [[epmrai*epmrai,ecovmu*epmrai*epmdeci], [ecovmu*epmrai*epmdeci,epmdeci*epmdeci]]
		g2pra, g2pde = np.random.multivariate_normal(mean, cov, Na).T
		g2pra=list(g2pra)
		g2pde=list(g2pde)

		mud=dataGCs['distance'][ff]
		sigmad=0.046*mud
		g1dd = np.random.normal(mud, sigmad, Na)
		g1dd=list(g1dd)

		muv=dataGCs['Vlos'][ff]
		sigmav=dataGCs['eVlos'][ff]
		g1dv = np.random.normal(muv, sigmav, Na)
		g1dv=list(g1dv)
		
		BBB=[]

		for gg in range(0,Na,1):

			di=random.choice(g1dd)
			vlosi=random.choice(g1dv)
			
			pmrai1=random.choice(g2pra)
			pmdeci1=random.choice(g2pde)
			
			g1dv.remove(vlosi)
			g1dd.remove(di)
			g2pra.remove(pmrai1)
			g2pde.remove(pmdeci1)


			BBB.append([Rai,Deci,di,pmrai1,pmdeci1,vlosi])

		
		my_critv = {}
		my_critr = {}
		my_candidates = {}

	#####Integration of GCs in MW+dwarfs orbit ##########################


		o= Orbit(BBB,radec=True)
		
		MWall= MWPotential2014 + my_dwarf_orbpots2['LMC'] + my_dwarf_orbpots['SMC'] + my_dwarf_orbpots['Sgr'] + my_dwarf_orbpots['Fornax'] + my_dwarf_orbpots['Carina'] + my_dwarf_orbpots['Leo II'] + my_dwarf_orbpots['Sextans'] + my_dwarf_orbpots['Sculptor'] + my_dwarf_orbpots['Draco'] + my_dwarf_orbpots['Umi'] + my_dwarf_orbpots['Leo I']
		
		o.integrate(ts,MWall + nip ,force_map=True)
    
		gcx=o.x(ts)
		gcy=o.y(ts)
		gcz=o.z(ts)
		vgcx=o.vx(ts)
		vgcy=o.vy(ts)
		vgcz=o.vz(ts)
		
		#gcE=o.E(ts)
		#gcL=o.L(ts)
		#gcLz=gcL[2]
		
		vgcx=o.vx(ts)
		vgcy=o.vy(ts)
		vgcz=o.vz(ts)
		Vgc=np.sqrt(vgcx**2 + vgcy**2 + vgcz**2) # Velocity of the GC in the dwarf reference frame


		######################## Finding associations between GCs and dwarfs based on the escape velocity and tidal radius criteria #############

		tab = ['LMC', 'SMC', 'Sgr', 'Fornax', 'Carina', 'Leo II', 'Sextans', 'Sculptor', 'Draco', 'Umi', 'Leo I']

		for i,name in enumerate(tab):
			
			if name=='LMC':
				dwarf1_orbit=my_dwarf_orbits2[name]
				dwarf1_pot=my_dwarf_pots2[name]
			else:
				dwarf1_orbit=my_dwarf_orbits[name]
				dwarf1_pot=my_dwarf_pots[name]
			X1=dwarf1_orbit.x(ts)
			Y1=dwarf1_orbit.y(ts)
			Z1=dwarf1_orbit.z(ts)
			R1=dwarf1_orbit.R(ts)
			VX1=dwarf1_orbit.vx(ts)
			VY1=dwarf1_orbit.vy(ts)
			VZ1=dwarf1_orbit.vz(ts)
					
			rr=dwarf1_orbit.r(ts)			
			En=dwarf1_orbit.E(ts)
			L=dwarf1_orbit.L(ts)
			Lz=L[2]
			
			############## Dynamical criteria ##########
			
			Rc=np.sqrt((gcx-X1)**2 + (gcy-Y1)**2 + (gcz-Z1)**2) # Orbital radius of GC centered on the dwarf
			Vdw=np.sqrt(VX1**2 + VY1**2 + VZ1**2) # Velocity of the GC in the dwarf reference frame
			Vrel=abs(Vgc-Vdw) # Velocity of the GC in the dwarf reference frame
			Vcir=dwarf1_pot.vcirc(Rc)
			Vesc=vesc(dwarf1_pot,Rc)
			rt=rtide(MWPotential2014 ,R1,Z1,M=Md0[i]*units.Msun) # tidal radius of the dwarf
			
			CritV=Vrel/Vesc # escape velocity criterion
			CritR=Rc/rt # tidal radius criterion
			Tranche=CritV*CritR
			Bingo=np.where(Tranche < 1)
			Nasso=np.array(Bingo).size
			if Nasso > 0:
				bingo0=list(set(Bingo[0]))
				LOC=[np.where(Bingo[0] == ki) for ki in bingo0]
				loc=[LOC[ll][0][0] for ll in range(0,len(bingo0),1)]
				indta=[Bingo[1][ll1] for ll1 in loc]
				critV=np.array([ CritV[mm][nn] for mm,nn in zip(bingo0,indta)])	
				critR=np.array([ CritR[mm][nn] for mm,nn in zip(bingo0,indta)])
				#dataE=np.array([ gcE[mm][nn] for mm,nn in zip(bingo0,indta)])
				#dataL=np.array([ gcL[mm][nn] for mm,nn in zip(bingo0,indta)])
				timeA=[ts1[ll2] for ll2 in indta]	
				critV1=np.where(critV < 1)
				critR1=np.where(critR < 1)
				Index1=critR1[0]
				NcritV=np.array(critV1).size
				NcritR=np.array(critR1).size
				#print('go')
				if NcritV>1 and NcritR>1:
					ListAt=[timeA[hh] for hh in Index1]
					print(len(ListAt)*100/Na,Average(ListAt),'GC:',name2,'Dwarf:',name)
					#ListE= [dataE[hh] for hh in Index1]
					#ListL= [dataL[hh] for hh in Index1]
					Number.append(ff)
					NAME.append(name2)
					PERCENT.append(len(ListAt)*100/Na)
					ACCRETTV.append(Average(ListAt))
					ACCRETT.append(ListAt)
	if len(Number)>0:			
		GO=np.array([Number,NAME,PERCENT,ACCRETTV,ACCRETT])			
		np.savetxt('GCR{}.txt'.format(kr),GO, fmt='%s')

# Affichage du temps d execution
print("Temps d execution : %s secondes ---" % (time.time() - start_time))

kr 0
moving lmc potential created
acceleration by LMC calculated
Dwarf number 1 /11
Dwarf number 2 /11
Dwarf number 3 /11
Dwarf number 4 /11
Dwarf number 5 /11
Dwarf number 6 /11
Dwarf number 7 /11
Dwarf number 8 /11
Dwarf number 9 /11
Dwarf number 10 /11
moving potentials for dwarfs created
